In [15]:
songTitle = input('Welcome to the Million Song Database Recommendation Engine (MSDRE). Enter the title of a song you would like recommendations for. i.e. Bodak Yellow: ')
artist = input('Enter the name of the artist i.e. Cardi B: ')

Welcome to the Million Song Database Recommendation Engine (MSDRE). Enter the title of a song you would like recommendations for. i.e. Bodak Yellow: Vordhosbn
Enter the name of the artist i.e. Cardi B: Aphex Twin


In [16]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id = "782cc49959664a40886e2583e1c0d429", client_secret = "446c0a76446f4f4f855e8e8337400328")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
results = sp.search(q= "artist:" + artist + " track:" + songTitle, type="track", offset = 0)
trackId = results['tracks']['items'][0]['id']
albumId = results['tracks']['items'][0]['id']
aAnal = sp.audio_analysis(trackId)

In [17]:
bars_confidence = aAnal['bars'][0]['confidence']
bars_start = aAnal['bars'][0]['start']
beats_confidence = aAnal['beats'][0]['start']
beats_start = aAnal['beats'][0]['start']
duration = aAnal['track']['duration']
end_of_fade_in = aAnal['track']['end_of_fade_in']
key_confidence = aAnal['track']['key_confidence']
key = aAnal['track']['key']
#latitude 
#longitude - both not part of Spotify API any longer
loudness = aAnal['track']['loudness']
mode = aAnal['track']['mode']
mode_confidence = aAnal['track']['mode_confidence']
start_of_fade_out = aAnal['track']['start_of_fade_out']
tatums_confidence = aAnal['tatums'][0]['confidence']
tatums_start = aAnal['tatums'][0]['start']
tempo = aAnal['track']['tempo']
time_signature = aAnal['track']['time_signature']
time_signature_confidence = aAnal['track']['time_signature_confidence']
year = results['tracks']['items'][0]['album']['release_date'][:4]

In [18]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Danny\Documents\spring18\DataScience\FinalProject\1000clusters.csv')
df = df.fillna(df.mean())
columnNames = ['bars_confidence','bars_start','beats_confidence','beats_start','duration','end_of_fade_in','key_confidence','key','loudness','mode','mode_confidence','start_of_fade_out','tatums_confidence','tatums_start','tempo','time_signature','time_signature_confidence','year']
y = df['clusters']
x= df[columnNames]
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
my_ANN = MLPClassifier(hidden_layer_sizes=(80,), activation= 'logistic', 
                       solver='adam', alpha=1e-5, random_state=1, 
                       learning_rate_init = 0.002)
my_ANN.fit(x, y)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(80,), learning_rate='constant',
       learning_rate_init=0.002, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [19]:
testSong = [bars_confidence,bars_start,beats_confidence,beats_start,duration,end_of_fade_in,key_confidence,key,loudness,mode,mode_confidence,start_of_fade_out,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,year]
pred = pd.DataFrame(data=[testSong])
pred.columns = ['bars_confidence','bars_start','beats_confidence','beats_start','duration','end_of_fade_in','key_confidence','key','loudness','mode','mode_confidence','start_of_fade_out','tatums_confidence','tatums_start','tempo','time_signature','time_signature_confidence','year']
ans = my_ANN.predict(pred)

In [20]:
dfCluster = df.loc[df['clusters'] == ans[0]]
artistId = results['tracks']['items'][0]['artists'][0]['id']


In [21]:
genres = sp.artist(artistId)['genres']

In [22]:
dfnew = dfCluster[['song.id','terms']]
dfnew['votes'] = 0

C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
taglist = genres[0].split()
for i in range(1, len(genres)):
    taglist = taglist + genres[i].split()

In [24]:
indices = dfnew.index.values
for i in range(len(dfnew)):
    term = dfnew.iloc[i]['terms']
    compare = term.split()
    for j in range(len(compare)):
        for k in range(len(taglist)):
            if(compare[j]==taglist[k]):
                dfnew.at[indices[i], 'votes'] = dfnew.iloc[i]['votes'] + 1

In [25]:
maxVotes = 0
for i in range(len(dfnew)):
    if (dfnew.iloc[i]['votes'] > maxVotes):
        maxVotes = dfnew.iloc[i]['votes']
reccs = []
for i in range(len(dfnew)):
    if (dfnew.iloc[i]['votes'] == maxVotes):
        reccs.append(dfnew.iloc[i]['song.id'])
        

In [26]:
for i in range(len(reccs)):
    dfAns = df.loc[df['song.id'] == reccs[i]]
    print(dfAns['title'])
    print(dfAns['artist.name'])

1283    Prayer For The Dying (Live In Paris)
Name: title, dtype: object
1283    Seal
Name: artist.name, dtype: object
2874    Still Love Remains (Album Version)
Name: title, dtype: object
2874    Seal
Name: artist.name, dtype: object


In [27]:
dfnew

,song.id,terms,votes
467,SOWODCW12A58A7AB81,death core,0
1027,SOKODRA12A8C13B4B5,folk rock,0
1283,SORLVXI12A8AE45E37,dance pop,1
1398,SOGRIWH12A8C1318AD,samba,0
1875,SOEXRTB12A8C139A38,ccm,0
1969,SOZSKYN12A8C13A883,country,0
2264,SOKQYBC12AB0183C87,blue-eyed soul,0
2348,SOWZREF12A8C13810B,instrumental rock,0
2391,SONCYFS12A8C14261E,viking metal,0
2545,SOXECRP12AB017EADA,urban,0


In [28]:
taglist

['acid',
 'techno',
 'ambient',
 'electronic',
 'fourth',
 'world',
 'intelligent',
 'dance',
 'music',
 'trip',
 'hop']